# Import Libraries and prepare resources

In [ ]:
import pandas as pd
import re, string
import numpy as np

import preprocessor as p

In [ ]:
import random, torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

# Read dataset

In [ ]:
path_train = "/kaggle/input/dataset-cs221/train_en.csv"
path_val = "/kaggle/input/dataset-cs221/val_en.csv"
path_test = "/kaggle/input/dataset-cs221/test_en.csv"

In [ ]:
# This is the simple preprocessing
# def preprocessing_text(text):
#     text = text.strip()
#     text = text.translate(text.maketrans('', '', string.punctuation.replace("_","")))
#     text = re.sub('\\s+',' ',text).strip()
#     return text

In [ ]:
# Empty preprocessing fucntion because the dataset used is preprocessed
def preprocessing_text(text):
    
    return text

## Preprocessing


In [ ]:
def convert_label(text):
    if text == "Hope":
        return 1
    else:
        return 0

def read_and_preprocessing(path_data):
    df = pd.read_csv(path_data)
    df["binary"] = df["binary"].apply(convert_label)
    x_input = df["text"].apply(preprocessing_text).tolist()
    y_output = df["binary"].tolist()
    ids = df["id"].tolist()
    return x_input,y_output,ids

train_texts, train_labels,train_ids = read_and_preprocessing(path_train)
valid_texts,valid_labels,valid_ids = read_and_preprocessing(path_val)
print(len(train_texts),len(train_labels))
print(len(valid_texts),len(valid_labels))

In [ ]:
df_train = pd.DataFrame(list(zip(train_texts, train_labels)),
               columns =['x_data', 'y_output'])
df_train.head()

# Build Filter Classifier

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "FacebookAI/xlm-roberta-base"

# bert_model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

bert_model = AutoModelForSequenceClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512, ignore_mismatched_sizes=True)

In [ ]:
max_length = 512
train_encodings = tokenizer(train_texts, truncation=True, max_length=max_length, padding=True)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    learning_rate=2e-5,              # 2e-5 for base models, 5e-5 for large model
    per_device_train_batch_size=32,  # batch size per device during training
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    save_total_limit = 1,
    report_to="tensorboard"
)

trainer = Trainer(
    model=bert_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)
trainer.train()

# Inference (Validation datasets)

In [ ]:
def make_prediction(review,tokenizer,trainer):
    demo_input = preprocessing_text(review)
    demo_encodings = tokenizer([demo_input], truncation=True, max_length = max_length, padding=True)
    test_dataset = CustomDataset(demo_encodings, [0])
    predic_demo = trainer.predict(test_dataset)[0]
    predict_label = np.argmax(predic_demo, axis=1).flatten().tolist()[0]
    return predict_label

In [ ]:
y_pred = []

for review in valid_texts:
    a = make_prediction(review,tokenizer,trainer)
    y_pred.append(a)
print(y_pred[:10])
print(valid_labels[:10])

In [ ]:
from sklearn.metrics import *

print("M_Pr: ", round(precision_score(valid_labels, y_pred, average='macro'),4))
print("M_Re: ", round(recall_score(valid_labels, y_pred, average='macro'),4))
print("M_F1: ", round(f1_score(valid_labels, y_pred, average='macro'),4))

print("W_Pr: ", round(precision_score(valid_labels, y_pred, average='weighted'),4))
print("W_Re: ", round(recall_score(valid_labels, y_pred, average='weighted'),4))
print("W_F1: ", round(f1_score(valid_labels, y_pred, average='weighted'),4))

print("acc:", round(accuracy_score(valid_labels, y_pred), 4))

# Predictions on Test datasets

In [ ]:
test_texts, test_labels,test_ids = read_and_preprocessing(path_test)

y_pred_test = []

for review in test_texts:
    a = make_prediction(review,tokenizer,trainer)
    y_pred.append(a)
print(y_pred_test[:10])
print(test_labels[:10])

In [ ]:
print("M_Pr: ", round(precision_score(test_labels, y_pred_test, average='macro'),4))
print("M_Re: ", round(recall_score(test_labels, y_pred_test, average='macro'),4))
print("M_F1: ", round(f1_score(valid_labels, y_pred, average='macro'),4))

print("W_Pr: ", round(precision_score(test_labels, y_pred_test, average='weighted'),4))
print("W_Re: ", round(recall_score(test_labels, y_pred_test, average='weighted'),4))
print("W_F1: ", round(f1_score(test_labels, y_pred_test, average='weighted'),4))

print("acc:", round(accuracy_score(test_labels, y_pred_test), 4))

## Convert back to label

In [ ]:
def convert2category(y_pred):
    y_label = []
    for y in y_pred:
        if y == 0:
              y_label.append("Not Hope")
        else:
              y_label.append("Hope")
    return y_label

In [ ]:
# name_sub = "predictions.csv"
# y_pred_label = convert2category(y_pred_test)
# df_sub = pd.DataFrame(list(zip(test_ids, y_pred_label)),
#                columns =['id', 'category'])
# df_sub.to_csv(name_sub)
# df_sub.head()